In [ ]:
# 1: Data Cleaning
# Cleaninig Prompt

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
# Importing Dataset

df = pd.read_excel(r"C:\Users\User\Desktop\Food_Delivery_Performance.xlsx")

# Making a copy of original dataset for cleaning
df_clean = df.copy()

In [ ]:
# Renaming columns for clarity
df_clean.rename(columns={
    "multiple_deliveries(yes_1, no_0)": "Multiple_deliveries",
    "Name:": "Unnamed"
}, inplace=True)

In [ ]:
# Handle missing values
df_clean.dropna(subset=[
    "Delivery_person_Age",
    "Delivery_person_Ratings",
    "Time_Orderd",
    "Weather",
    "Road_traffic_density",
    "City"
], inplace=True)

In [ ]:
# Fill 'Festival' with 'No' assuming missing implies non-festival
df_clean["Festival"].fillna("No", inplace=True)

In [ ]:
# Fill missing 'Multiple_deliveries' with 0 (assuming no multiple delivery)
df_clean["Multiple_deliveries"].fillna(0, inplace=True)

In [ ]:
df_clean.info()

In [ ]:
# Convert data types
df_clean["Delivery_person_Age"] = df_clean["Delivery_person_Age"].astype(int)
df_clean["Multiple_deliveries"] = df_clean["Multiple_deliveries"].astype(int)
df_clean["Order_Date"] = pd.to_datetime(df_clean["Order_Date"], errors='coerce', dayfirst=True)
df_clean["Time_Orderd"] = pd.to_datetime(df_clean["Time_Orderd"], format='%H:%M:%S', errors='coerce').dt.time
df_clean["Time_Order_picked"] = pd.to_datetime(df_clean["Time_Order_picked"], format='%H:%M:%S', errors='coerce').dt.time

In [ ]:
# Drop 'Unnamed' column if it doesn't provide useful info
df_clean.drop(columns=["Unnamed"], inplace=True)

In [ ]:
# Remove outliers in numeric columns (IQR method for age and ratings)
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    return data[(data[column] >= Q1 - 1.5 * IQR) & (data[column] <= Q3 + 1.5 * IQR)]

In [ ]:
df_clean = remove_outliers_iqr(df_clean, "Delivery_person_Age")
df_clean = remove_outliers_iqr(df_clean, "Delivery_person_Ratings")

In [ ]:
# Summary of cleaned data
df_clean.info(), df_clean.head()

In [ ]:
# 2: Descriptive Statistics for key numerical variables

numeric_cols = [
    "Delivery_person_Age",
    "Delivery_person_Ratings",
    "Restaurant_latitude",
    "Restaurant_longitude",
    "Delivery_location_latitude",
    "Delivery_location_longitude",
    "Vehicle_condition",
    "Multiple_deliveries"
]

In [ ]:
# Generate descriptive statistics
desc_stats = df_clean[numeric_cols].describe().T  # Transpose for better readability

desc_stats

In [ ]:
df_clean

In [ ]:
# Delivery Time Analysis
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Combine Order_Date + Time_Orderd into full datetime
df_clean['Order_DateTime_Orderd'] = pd.to_datetime(df_clean['Order_Date'].astype(str) + ' ' + df_clean['Time_Orderd'].astype(str), errors='coerce')
df_clean['Order_DateTime_Picked'] = pd.to_datetime(df_clean['Order_Date'].astype(str) + ' ' + df_clean['Time_Order_picked'].astype(str), errors='coerce')

In [ ]:
# Calculate Delivery Duration in minutes
df_clean['Delivery_duration_mins'] = (df_clean['Order_DateTime_Picked'] - df_clean['Order_DateTime_Orderd']).dt.total_seconds() / 60

In [ ]:
# Drop any negative or extremely large durations
df_clean = df_clean[(df_clean['Delivery_duration_mins'] > 0) & (df_clean['Delivery_duration_mins'] < 300)]

In [ ]:
# Analyzing average delivery time under different conditions

# 1. By Traffic Density
traffic_delivery = df_clean.groupby('Road_traffic_density')['Delivery_duration_mins'].mean().sort_values()

# 2. By Weather Conditions
weather_delivery = df_clean.groupby('Weather')['Delivery_duration_mins'].mean().sort_values()

# 3. By Vehicle Type
vehicle_delivery = df_clean.groupby('Type_of_vehicle')['Delivery_duration_mins'].mean().sort_values()

In [ ]:
# Display results
print("🚦 Average Delivery Time by Traffic Density:")
print(traffic_delivery)
print("\n☁️ Average Delivery Time by Weather:")
print(weather_delivery)
print("\n🏍️ Average Delivery Time by Vehicle Type:")
print(vehicle_delivery)

In [ ]:
# Plotting Average Delivery Time Across Different Conditions
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(18, 5))

In [ ]:
# 1️⃣ Plot Average Delivery Time by Traffic Density
plt.subplot(1, 3, 1)
sns.barplot(x=traffic_delivery.index, y=traffic_delivery.values, palette="coolwarm")
plt.title("🚦 Avg Delivery Time by Traffic Density", fontsize=14)
plt.xlabel("Traffic Density")
plt.ylabel("Average Delivery Time (minutes)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2️⃣ Plot Average Delivery Time by Weather
plt.subplot(1, 3, 2)
sns.barplot(x=weather_delivery.index, y=weather_delivery.values, palette="Blues_r")
plt.title("☁️ Avg Delivery Time by Weather", fontsize=14)
plt.xlabel("Weather")
plt.ylabel("")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3️⃣ Plot Average Delivery Time by Vehicle Type
plt.subplot(1, 3, 3)
sns.barplot(x=vehicle_delivery.index, y=vehicle_delivery.values, palette="Greens_r")
plt.title("🏍️ Avg Delivery Time by Vehicle Type", fontsize=14)
plt.xlabel("Vehicle Type")
plt.ylabel("")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# 4: Effect Of Traffic Density

sns.boxplot(
    data=df_clean,
    x='Road_traffic_density',
    y='Delivery_duration_mins',
    palette="Set2")

In [ ]:
# Add title and labels
plt.title("🚦 Delivery Time Distribution by Traffic Density", fontsize=16)
plt.xlabel("Traffic Density", fontsize=12)
plt.ylabel("Delivery Duration (minutes)", fontsize=12)
plt.xticks(rotation=45)

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
# 5: Effect Of Weather Condition

sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))

In [ ]:
# Create a boxplot for delivery time vs weather
sns.boxplot(
    data=df_clean,
    x='Weather',
    y='Delivery_duration_mins',
    palette="coolwarm")

In [ ]:
# Add titles and labels
plt.title("🌦️ Delivery Time Distribution by Weather Condition", fontsize=16)
plt.xlabel("Weather Condition", fontsize=12)
plt.ylabel("Delivery Duration (minutes)", fontsize=12)
plt.xticks(rotation=45)

In [ ]:
# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# --------------- ⭐ Analysis by Delivery Personnel Ratings ---------------

# Bin ratings into categories
rating_bins = [0, 2.5, 3.5, 4.5, 5.0]
rating_labels = ['Low', 'Medium', 'High', 'Excellent']
df_clean['Rating_category'] = pd.cut(df_clean['Delivery_person_Ratings'], bins=rating_bins, labels=rating_labels)

In [ ]:
# Plot average delivery duration by rating category
plt.figure(figsize=(8, 5))
sns.barplot(data=df_clean, x='Rating_category', y='Delivery_duration_mins', palette='YlGnBu')
plt.title('⭐ Avg Delivery Time by Delivery Personnel Ratings', fontsize=16)
plt.xlabel('Rating Category', fontsize=12)
plt.ylabel('Average Delivery Duration (minutes)', fontsize=12)
plt.show()

In [ ]:
# --------------- 👨‍🦳 Analysis by Delivery Personnel Age ---------------

# Bin ages into groups
age_bins = [17, 25, 35, 50, 65]
age_labels = ['18-25', '26-35', '36-50', '51-65']
df_clean['Age_group'] = pd.cut(df_clean['Delivery_person_Age'], bins=age_bins, labels=age_labels)

In [ ]:

# Plot average delivery duration by age group
plt.figure(figsize=(8, 5))
sns.barplot(data=df_clean, x='Age_group', y='Delivery_duration_mins', palette='Oranges')
plt.title('👨‍🦳 Avg Delivery Time by Delivery Personnel Age Group', fontsize=16)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Average Delivery Duration (minutes)', fontsize=12)
plt.show()

In [ ]:
#Task 7
# First, ensure Festival column has consistent format
df_clean['Festival'] = df_clean['Festival'].str.strip().str.lower()

In [ ]:
# Group by Festival status
festival_summary = df_clean.groupby('Festival').agg({
    'Delivery_duration_mins': 'mean',
    'ID': 'count'   # Count of orders
}).rename(columns={'ID': 'Order_Count'})

In [ ]:
print("🎉 Festival Impact Summary:")
print(festival_summary)

In [ ]:

# --------------- 📊 Visualization: Delivery Time ---------------

plt.figure(figsize=(8, 5))
sns.barplot(data=df_clean, x='Festival', y='Delivery_duration_mins', palette='pastel')
plt.title('🎈 Average Delivery Time During Festival vs Non-Festival', fontsize=16)
plt.xlabel('Festival (yes or no)', fontsize=12)
plt.ylabel('Average Delivery Duration (minutes)', fontsize=12)
plt.xticks([0, 1], ['No', 'Yes'])  # assuming 'no', 'yes' in lowercase
plt.show()

In [ ]:
# --------------- 📊 Visualization: Order Volume ---------------

plt.figure(figsize=(8, 5))
sns.countplot(data=df_clean, x='Festival', palette='pastel')
plt.title('🎉 Order Volume During Festival vs Non-Festival', fontsize=16)
plt.xlabel('Festival (yes or no)', fontsize=12)
plt.ylabel('Order Count', fontsize=12)
plt.xticks([0, 1], ['No', 'Yes'])  
plt.show()

In [ ]:
pip install folium

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
# --------------- 🗺️ Create a map for Delivery Locations ---------------

# 1. Find the center point for the initial map
latitude_center = df_clean['Delivery_location_latitude'].mean()
longitude_center = df_clean['Delivery_location_longitude'].mean()


In [ ]:
# 2. Initialize the Folium map
delivery_map = folium.Map(location=[latitude_center, longitude_center], zoom_start=12)

In [ ]:
# 3. Add a Marker Cluster
marker_cluster = MarkerCluster().add_to(delivery_map)

In [ ]:
# 4. Add each delivery point to the cluster
for idx, row in df_clean.iterrows():
    folium.Marker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        popup=f"Delivery ID: {row['ID']}",
        icon=folium.Icon(color='blue', icon='bicycle', prefix='fa')).add_to(marker_cluster)

In [ ]:
# 5. Display the map
delivery_map

In [ ]:
import folium
from folium.plugins import MarkerCluster

# --------------- 🗺️ Create a map for Restaurant and Delivery Locations ---------------

# 1. Center point for the map
latitude_center = df_clean[['Restaurant_latitude', 'Delivery_location_latitude']].mean().mean()
longitude_center = df_clean[['Restaurant_longitude', 'Delivery_location_longitude']].mean().mean()

# 2. Initialize the Folium map
full_map = folium.Map(location=[latitude_center, longitude_center], zoom_start=12)

# 3. Marker Clusters for Restaurants and Deliveries
restaurant_cluster = MarkerCluster(name='Restaurants').add_to(full_map)
delivery_cluster = MarkerCluster(name='Deliveries').add_to(full_map)

# 4. Add Restaurant Locations (red icons)
for idx, row in df_clean.iterrows():
    folium.Marker(
        location=[row['Restaurant_latitude'], row['Restaurant_longitude']],
        popup=f"Restaurant for ID: {row['ID']}",
        icon=folium.Icon(color='red', icon='cutlery', prefix='fa')
    ).add_to(restaurant_cluster)

# 5. Add Delivery Locations (blue icons)
for idx, row in df_clean.iterrows():
    folium.Marker(
        location=[row['Delivery_location_latitude'], row['Delivery_location_longitude']],
        popup=f"Delivery to ID: {row['ID']}",
        icon=folium.Icon(color='blue', icon='shopping-bag', prefix='fa')
    ).add_to(delivery_cluster)

# 6. Add a layer control toggle
folium.LayerControl().add_to(full_map)

# 7. Show the map
full_map

In [ ]:
# 9

In [ ]:
# --📅 Daily Order Trends --

# First, make sure Order_Date is a proper datetime
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'])

# Group by order date and count number of orders
daily_orders = df_clean.groupby(df_clean['Order_Date']).size()

# Plot daily orders
plt.figure(figsize=(12, 6))
daily_orders.plot(kind='line', color='teal')
plt.title('📅 Daily Order Trend', fontsize=16)
plt.xlabel('Order Date', fontsize=12)
plt.ylabel('Number of Orders', fontsize=12)
plt.grid(True)
plt.show()

# -- ⏰ Hourly Order Trends --

# Extract hour from Time_Orderd
df_clean['Hour_Orderd'] = pd.to_datetime(df_clean['Time_Orderd'], format='%H:%M:%S', errors='coerce').dt.hour

# Group by hour and count number of orders
hourly_orders = df_clean.groupby('Hour_Orderd').size()

# Plot hourly orders
plt.figure(figsize=(12, 6))
sns.barplot(x=hourly_orders.index, y=hourly_orders.values, palette='viridis')
plt.title('⏰ Hourly Order Trend', fontsize=16)
plt.xlabel('Hour of the Day', fontsize=12)
plt.ylabel('Number of Orders', fontsize=12)
plt.xticks(range(0,24))
plt.grid(True)
plt.show()

In [ ]:
# 10

In [ ]:
# --------------- 🚗 Vehicle Condition Analysis ---------------

# Quick look at the unique values
print("Unique Vehicle Condition Scores:", df_clean['Vehicle_condition'].unique())

# Plot delivery duration vs vehicle condition
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=df_clean,
    x='Vehicle_condition',
    y='Delivery_duration_mins',
    palette='Set3'
)

plt.title('🚗 Delivery Time Distribution by Vehicle Condition', fontsize=16)
plt.xlabel('Vehicle Condition Score (Higher = Better)', fontsize=12)
plt.ylabel('Delivery Duration (minutes)', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
vehicle_condition_summary = df_clean.groupby('Vehicle_condition')['Delivery_duration_mins'].mean()
print(vehicle_condition_summary)

In [ ]:
# 11

In [ ]:
print(df_clean.columns.tolist())

In [ ]:
# --------------- 📦 Multiple Deliveries Analysis ---------------

# Quick check on unique delivery counts
print("Unique Multiple Deliveries Values:", df_clean['Multiple_deliveries'].unique())

# Make sure multiple_deliveries is numeric
df_clean['Multiple_deliveries'] = pd.to_numeric(df_clean['Multiple_deliveries'], errors='coerce')

# Plot delivery duration vs number of deliveries
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df_clean,
    x='Multiple_deliveries',
    y='Delivery_duration_mins',
    palette='Accent')

plt.title('📦➕ Delivery Time vs Number of Multiple Deliveries', fontsize=16)
plt.xlabel('Number of Deliveries (per trip)', fontsize=12)
plt.ylabel('Delivery Duration (minutes)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
df_clean.to_csv('clean_food_delivery_data.csv', index=False)

In [ ]:
df_clean.head(50)